<a href="https://colab.research.google.com/github/p25-c4/plate_project/blob/main/scale_train_test_split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [52]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
df_raw = pd.read_csv("/content/drive/MyDrive/BigData/final_scale.csv")
df_raw.head()

,scale,spec_long,spec_country,steel_kind,pt_thick,pt_width,pt_length,hsb,fur_no,fur_input_row,fur_heat_temp,fur_heat_time,fur_soak_temp,fur_soak_time,fur_total_time,rolling_method,rolling_temp,descaling_count,day_yn,purpose
0,0,AB/EH32-TM,미국,T,32,3700,15100,적용,1,1,1144,116,1133,59,259,TMCP(온도제어),934,8,주간,선체구조용(조선용)
1,0,AB/EH32-TM,미국,T,32,3700,15100,적용,1,2,1144,122,1135,53,238,TMCP(온도제어),937,8,주간,선체구조용(조선용)
2,0,NV-E36-TM,영국,T,33,3600,19200,적용,2,1,1129,116,1121,55,258,TMCP(온도제어),889,8,주간,선체구조용(조선용)
3,0,NV-E36-TM,영국,T,33,3600,19200,적용,2,2,1152,125,1127,68,266,TMCP(온도제어),885,8,주간,선체구조용(조선용)
4,0,BV-EH36-TM,프랑스,T,38,3100,13300,적용,3,1,1140,134,1128,48,246,TMCP(온도제어),873,8,주간,선체구조용(조선용)


In [54]:
df_raw_x_1 = df_raw.drop("scale", axis=1)
df_raw_x_2 = df_raw[['steel_kind', 'day_yn', 'rolling_method']]  # 범주가 2개인 자료는 따로 dummy화
df_raw_y = df_raw['scale']

In [56]:

# 범주형 변수 선택 : select_dtypes="object"
df_raw_x_char_1 = df_raw_x_1.select_dtypes(include = "object")
df_raw_x_char_2 = df_raw_x_2.select_dtypes(include = "object")

# get_dummies: 범주형 변수에 대한 더미변환
df_x_dummy_1 = pd.get_dummies(df_raw_x_char_1)
df_x_dummy_2 = pd.get_dummies(df_raw_x_char_2, drop_first=True) # 범주가 2개인 자료는 따로 dummy화
df_x_dummy = pd.concat([df_x_dummy_1, df_x_dummy_2], axis=1)


# 연속형 변수 선택 : select_dtypes="object" 제외
df_raw_x_num = df_raw_x.select_dtypes(exclude = "object")

# 변수명 저장
v_feature_names = df_raw_x_num.columns

# StandardScaler 적용
scaler = StandardScaler()
df_x_scaled = scaler.fit_transform(df_raw_x_num)
df_x_scaled = pd.DataFrame(df_x_scaled, columns=v_feature_names)

# 원래 연속형(df_raw_x_num) + dummy된 범주형(df_x_dummy)
df_x_raw_dummy = df_raw_x_num.join(df_x_dummy)
display("원래 연속형+ Dummied 범주형...DT/RF/GB 용:",df_x_raw_dummy.head(3))

# scale된 연속형(df_x_scaled)+ dummy된 범주형(df_x_dummy)
df_x_scale_dummy = df_x_scaled.join(df_x_dummy)
display("Scaled 연속형+ Dummied 범주형...SVM/NN/KNN 용:",df_x_scale_dummy.head(3))

# scale된 연속형(df_x_scaled) + 원래 범주형(df_raw_x_char)
df_x_scale_raw = df_x_scaled.join(df_raw_x_char)
display("Scaled 연속형+ 원래 범주형...회귀분석용(표준화 회귀계수):",df_x_scale_raw.head(3))

'원래 연속형+ Dummied 범주형...DT/RF/GB 용:'

,pt_thick,pt_width,pt_length,fur_no,fur_input_row,fur_heat_temp,fur_heat_time,fur_soak_temp,fur_soak_time,fur_total_time,...,rolling_method_TMCP(온도제어),day_yn_야간,day_yn_주간,purpose_OTHERS,purpose_선체구조용(조선용),purpose_용접구조용,purpose_일반구조용,steel_kind_T,day_yn_주간,rolling_method_TMCP(온도제어)
0,32,3700,15100,1,1,1144,116,1133,59,259,...,1,0,1,0,1,0,0,1,1,1
1,32,3700,15100,1,2,1144,122,1135,53,238,...,1,0,1,0,1,0,0,1,1,1
2,33,3600,19200,2,1,1129,116,1121,55,258,...,1,0,1,0,1,0,0,1,1,1


'Scaled 연속형+ Dummied 범주형...SVM/NN/KNN 용:'

,pt_thick,pt_width,pt_length,fur_no,fur_input_row,fur_heat_temp,fur_heat_time,fur_soak_temp,fur_soak_time,fur_total_time,...,rolling_method_TMCP(온도제어),day_yn_야간,day_yn_주간,purpose_OTHERS,purpose_선체구조용(조선용),purpose_용접구조용,purpose_일반구조용,steel_kind_T,day_yn_주간,rolling_method_TMCP(온도제어)
0,0.298839,1.765880,-1.561273,-1.227015,-0.982207,-0.623951,1.13104,-1.034421,-0.615879,0.534990,...,1,0,1,0,1,0,0,1,1,1
1,0.298839,1.765880,-1.561273,-1.227015,1.018115,-0.623951,1.35857,-0.918753,-0.906940,-0.017416,...,1,0,1,0,1,0,0,1,1,1
2,0.354957,1.562785,-1.265674,-0.003875,-0.982207,-1.333010,1.13104,-1.728433,-0.809920,0.508684,...,1,0,1,0,1,0,0,1,1,1


'Scaled 연속형+ 원래 범주형...회귀분석용(표준화 회귀계수):'

,pt_thick,pt_width,pt_length,fur_no,fur_input_row,fur_heat_temp,fur_heat_time,fur_soak_temp,fur_soak_time,fur_total_time,rolling_temp,descaling_count,spec_country,steel_kind,rolling_method,night_yn,purpose
0,0.298839,1.765880,-1.561273,-1.227015,-0.982207,-0.623951,1.13104,-1.034421,-0.615879,0.534990,-0.103218,-0.342411,미국,T,TMCP(온도제어),주간,선체구조용(조선용)
1,0.298839,1.765880,-1.561273,-1.227015,1.018115,-0.623951,1.35857,-0.918753,-0.906940,-0.017416,-0.056516,-0.342411,미국,T,TMCP(온도제어),주간,선체구조용(조선용)
2,0.354957,1.562785,-1.265674,-0.003875,-0.982207,-1.333010,1.13104,-1.728433,-0.809920,0.508684,-0.803747,-0.342411,영국,T,TMCP(온도제어),주간,선체구조용(조선용)


In [49]:
df_x_raw_dummy

,pt_thick,pt_width,pt_length,fur_no,fur_input_row,fur_heat_temp,fur_heat_time,fur_soak_temp,fur_soak_time,fur_total_time,...,rolling_method_TMCP(온도제어),day_yn_야간,day_yn_주간,purpose_OTHERS,purpose_선체구조용(조선용),purpose_용접구조용,purpose_일반구조용,steel_kind_T,day_yn_주간,rolling_method_TMCP(온도제어)
0,32,3700,15100,1,1,1144,116,1133,59,259,...,1,0,1,0,1,0,0,1,1,1
1,32,3700,15100,1,2,1144,122,1135,53,238,...,1,0,1,0,1,0,0,1,1,1
2,33,3600,19200,2,1,1129,116,1121,55,258,...,1,0,1,0,1,0,0,1,1,1
3,33,3600,19200,2,2,1152,125,1127,68,266,...,1,0,1,0,1,0,0,1,1,1
4,38,3100,13300,3,1,1140,134,1128,48,246,...,1,0,1,0,1,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
942,19,3400,41500,1,2,1142,55,1151,86,203,...,0,1,0,0,1,0,0,0,0,0
943,19,3400,41500,1,2,1142,55,1151,86,209,...,0,1,0,0,1,0,0,0,0,0
944,17,3400,43700,2,2,1169,65,1163,77,227,...,0,1,0,0,1,0,0,0,0,0
945,17,3400,43700,2,2,1169,65,1163,77,204,...,0,1,0,0,1,0,0,0,0,0


In [50]:
## train_test_split
# 원래 연속형+ Dummied 범주형...DT/RF/GB 용:df_x_raw_dummy
df_train_x_rd, df_test_x_rd, df_train_y, df_test_y = train_test_split(df_x_raw_dummy, df_raw_y
                                    , test_size = 0.2, stratify = df_x_raw_dummy["steel_kind_T"])

display("원래 연속형+ Dummied 범주형...DT/RF/GB 용:", df_train_x_rd.head())

# 기계학습용 데이터(Scaled 연속형+ Dummied 범주형) 분할:df_x_scale_dummy
df_train_x_sd, df_test_x_sd, df_train_y, df_test_y = train_test_split(df_x_scale_dummy, df_raw_y
                                    , test_size = 0.2, stratify = df_x_raw_dummy["steel_kind_T"])

display("Scaled 연속형+ Dummied 범주형...SVM/NN/KNN 용:", df_train_x_sd.head())

# 회귀분석용(표준화 회귀계수) 데이터(Scaled 연속형+ 원래 범주형) 분할:df_x_scale_raw
df_train_x_reg, df_test_x_reg, df_train_y, df_test_y = train_test_split(df_x_scale_raw, df_raw_y
                                    , test_size = 0.2, stratify = df_x_raw_dummy["steel_kind_T"])

display("Scaled 연속형+ 원래 범주형...회귀분석용(표준화 회귀계수):",df_train_x_reg.head())

'원래 연속형+ Dummied 범주형...DT/RF/GB 용:'

,pt_thick,pt_width,pt_length,fur_no,fur_input_row,fur_heat_temp,fur_heat_time,fur_soak_temp,fur_soak_time,fur_total_time,...,rolling_method_TMCP(온도제어),day_yn_야간,day_yn_주간,purpose_OTHERS,purpose_선체구조용(조선용),purpose_용접구조용,purpose_일반구조용,steel_kind_T,day_yn_주간,rolling_method_TMCP(온도제어)
919,12,2900,53300,2,2,1160,69,1160,65,217,...,0,1,0,0,1,0,0,0,0,0
913,12,2900,52900,3,1,1185,79,1163,58,207,...,0,1,0,0,1,0,0,0,0,0
826,12,2800,54500,2,2,1171,71,1162,67,231,...,0,0,1,0,1,0,0,0,1,0
683,23,2500,46400,3,1,1163,126,1158,60,242,...,0,0,1,0,1,0,0,0,1,0
416,14,3500,43000,3,1,1189,63,1175,60,215,...,0,0,1,0,1,0,0,0,1,0


'Scaled 연속형+ Dummied 범주형...SVM/NN/KNN 용:'

,pt_thick,pt_width,pt_length,fur_no,fur_input_row,fur_heat_temp,fur_heat_time,fur_soak_temp,fur_soak_time,fur_total_time,...,rolling_method_TMCP(온도제어),day_yn_야간,day_yn_주간,purpose_OTHERS,purpose_선체구조용(조선용),purpose_용접구조용,purpose_일반구조용,steel_kind_T,day_yn_주간,rolling_method_TMCP(온도제어)
807,-0.430688,-1.686744,0.147430,-0.003875,-0.982207,-0.529410,-0.196215,0.006596,-0.518859,-0.806568,...,0,1,0,0,0,1,0,0,0,0
451,-0.711276,0.953498,1.041435,-0.003875,-0.982207,1.408683,-0.651274,0.931944,-1.246512,-0.543517,...,0,0,1,0,0,1,0,0,1,0
469,-0.542923,0.547307,0.969337,1.219266,1.018115,2.212282,-0.575431,1.452453,-1.052471,-0.990703,...,0,0,1,0,1,0,0,0,1,0
249,0.747779,-1.280553,-0.436557,1.219266,1.018115,0.037836,-0.499588,0.295767,0.936446,0.824345,...,0,1,0,0,0,1,0,0,0,0
789,-0.711276,-0.468171,1.250516,1.219266,1.018115,-0.056705,-0.309980,0.989778,0.208794,0.429769,...,0,1,0,0,1,0,0,0,0,0


'Scaled 연속형+ 원래 범주형...회귀분석용(표준화 회귀계수):'

,pt_thick,pt_width,pt_length,fur_no,fur_input_row,fur_heat_temp,fur_heat_time,fur_soak_temp,fur_soak_time,fur_total_time,rolling_temp,descaling_count,spec_country,steel_kind,rolling_method,night_yn,purpose
427,-0.655158,0.953498,0.226737,-1.227015,-0.982207,1.975930,-0.499588,1.857293,-0.809920,-0.753957,0.176994,0.909569,영국,C,CR(제어압연),주간,선체구조용(조선용)
146,-0.711276,-0.265075,1.236097,-0.003875,-0.982207,0.368730,1.851550,0.758441,-0.518859,0.640210,0.410504,0.909569,한국,C,CR(제어압연),주간,용접구조용
857,-0.374571,0.547307,-0.378879,-0.003875,-0.982207,0.935977,-0.575431,1.221116,-1.052471,-0.964398,0.830822,-0.342411,미국,C,CR(제어압연),주간,용접구조용
529,0.747779,-1.077457,-0.811462,1.219266,1.018115,0.179648,0.713903,-1.497096,2.440262,2.271122,-1.877892,-1.594390,공통,T,CR(제어압연),야간,OTHERS
452,-0.711276,0.953498,1.185629,-1.227015,-0.982207,1.125059,1.244805,1.336784,-0.906940,0.561295,-0.414564,0.909569,미국,C,CR(제어압연),주간,선체구조용(조선용)
